In [8]:
import os
import talib
import logging
import pandas as pd
from datetime import datetime

logging.basicConfig(level=logging.INFO, format="%(asctime)s [%(levelname)s] %(message)s")
root_path = "/Users/fang/stock_data/basic_data"
# root_path = "C:\\Users\\Fang\\PycharmProjects\\股票分析\\stock_data\\basic_data"
# root_path = "C:\\Users\\Flora\\Documents\\股票分析\\stock_data\\basic_data"

In [16]:
df_list = []
start_dt = '2020-01-01'
end_dt = '2020-07-15'
bband_slope_change_criteria = 0.5
bband_width_change_criteria = 0.25
prev_bband_width_criteria = 5
for file in sorted(os.listdir(root_path)):
    if '.csv' not in file:
        continue
    logging.info(file.replace('.csv', ''))
    
    # load stock price data
    file_path = root_path + os.sep + file
    try:
        stock_df = pd.read_csv(file_path, index_col=None, header=0, dtype={'股票代號':str}, engine='python', encoding='utf-8').sort_values('日期')
    except:
        stock_df = pd.read_csv(file_path, index_col=None, header=0, dtype={'股票代號':str}, engine='python').sort_values('日期')
    stock_df.loc[:, '日期'] = pd.to_datetime(stock_df['日期'].str.replace('＊', ''))
    stock_df = stock_df[(stock_df['日期'] >= start_dt) & (stock_df['日期'] <= end_dt)]
    if len(stock_df) < 60:
        continue
    stock_df = stock_df.sort_values('日期')
    stock_df['收盤價'].fillna(method='ffill', inplace=True)
    
    # calculate BBands and some features
    closed = stock_df['收盤價'].values
    upper,middle,lower = talib.BBANDS(closed,20, 2, 2, matype=talib.MA_Type.SMA)
    previous_upper = pd.Series(upper).shift(periods=1).to_numpy()
    previous_lower = pd.Series(lower).shift(periods=1).to_numpy()
    upper_change = pd.Series((upper - previous_upper) / previous_upper*100)
    lower_change = (lower - previous_lower) / previous_lower*100
    bband_width =  pd.Series((upper-lower) / middle*100)
    bband_width_ma = pd.Series(talib.SMA(bband_width, 5))
    
    # add BBands data to stock price dataframe
    stock_df = stock_df.reset_index().drop('index', axis=1)
    stock_df['bband_width'] = bband_width
    stock_df['prev_bband_width'] = stock_df['bband_width'].shift(periods=1)
    stock_df['bband_slope'] = upper_change
    stock_df['prev_bband_slope'] = stock_df['bband_slope'].shift(periods=1)
    stock_df['bband_slope_change'] = abs(stock_df['bband_slope'] - stock_df['prev_bband_slope']) / (abs(stock_df['prev_bband_slope'] + 0.00000001))
    stock_df['bband_width_change'] = (stock_df['bband_width'] - stock_df['prev_bband_width']) / (stock_df['prev_bband_width'] + 0.00000001)
    
    # filter data if meet the alert conditions
    filtered_df = stock_df[(stock_df['bband_slope_change']>bband_slope_change_criteria) 
                           & (stock_df['bband_width_change']>bband_width_change_criteria) 
                           & (stock_df['prev_bband_width']<prev_bband_width_criteria)]
    df_list.append(filtered_df)
alert_df = pd.concat(df_list, axis=0, ignore_index=True, sort=False)

2020-07-15 20:29:33,164 [INFO] 0050
2020-07-15 20:29:33,248 [INFO] 0051
2020-07-15 20:29:33,282 [INFO] 0052
2020-07-15 20:29:33,315 [INFO] 0053
2020-07-15 20:29:33,349 [INFO] 0054
2020-07-15 20:29:33,381 [INFO] 0055
2020-07-15 20:29:33,431 [INFO] 0056
2020-07-15 20:29:33,462 [INFO] 0057
2020-07-15 20:29:33,495 [INFO] 0058
2020-07-15 20:29:33,507 [INFO] 0059
2020-07-15 20:29:33,537 [INFO] 0061
2020-07-15 20:29:33,582 [INFO] 1101
2020-07-15 20:29:33,626 [INFO] 1102
2020-07-15 20:29:33,671 [INFO] 1103
2020-07-15 20:29:33,714 [INFO] 1104
2020-07-15 20:29:33,758 [INFO] 1108
2020-07-15 20:29:33,803 [INFO] 1109
2020-07-15 20:29:33,847 [INFO] 1110
2020-07-15 20:29:33,882 [INFO] 1201
2020-07-15 20:29:33,942 [INFO] 1203
2020-07-15 20:29:33,974 [INFO] 1210
2020-07-15 20:29:34,012 [INFO] 1213
2020-07-15 20:29:34,043 [INFO] 1215
2020-07-15 20:29:34,074 [INFO] 1216
2020-07-15 20:29:34,115 [INFO] 1217
2020-07-15 20:29:34,157 [INFO] 1218
2020-07-15 20:29:34,197 [INFO] 1219
2020-07-15 20:29:34,240 [INF

2020-07-15 20:29:41,474 [INFO] 1795
2020-07-15 20:29:41,496 [INFO] 1796
2020-07-15 20:29:41,519 [INFO] 1799
2020-07-15 20:29:41,540 [INFO] 1802
2020-07-15 20:29:41,562 [INFO] 1805
2020-07-15 20:29:41,585 [INFO] 1806
2020-07-15 20:29:41,608 [INFO] 1808
2020-07-15 20:29:41,629 [INFO] 1809
2020-07-15 20:29:41,651 [INFO] 1810
2020-07-15 20:29:41,674 [INFO] 1813
2020-07-15 20:29:41,695 [INFO] 1815
2020-07-15 20:29:41,717 [INFO] 1817
2020-07-15 20:29:41,739 [INFO] 1902
2020-07-15 20:29:41,765 [INFO] 1903
2020-07-15 20:29:41,788 [INFO] 1904
2020-07-15 20:29:41,809 [INFO] 1905
2020-07-15 20:29:41,831 [INFO] 1906
2020-07-15 20:29:41,855 [INFO] 1907
2020-07-15 20:29:41,882 [INFO] 1909
2020-07-15 20:29:41,909 [INFO] 2002
2020-07-15 20:29:41,934 [INFO] 2006
2020-07-15 20:29:41,957 [INFO] 2007
2020-07-15 20:29:41,981 [INFO] 2008
2020-07-15 20:29:42,006 [INFO] 2009
2020-07-15 20:29:42,030 [INFO] 2010
2020-07-15 20:29:42,055 [INFO] 2012
2020-07-15 20:29:42,080 [INFO] 2013
2020-07-15 20:29:42,103 [INF

2020-07-15 20:29:48,602 [INFO] 2511
2020-07-15 20:29:48,671 [INFO] 2514
2020-07-15 20:29:48,711 [INFO] 2515
2020-07-15 20:29:48,757 [INFO] 2516
2020-07-15 20:29:48,796 [INFO] 2520
2020-07-15 20:29:48,845 [INFO] 2524
2020-07-15 20:29:48,927 [INFO] 2527
2020-07-15 20:29:48,973 [INFO] 2528
2020-07-15 20:29:49,024 [INFO] 2530
2020-07-15 20:29:49,074 [INFO] 2534
2020-07-15 20:29:49,113 [INFO] 2535
2020-07-15 20:29:49,156 [INFO] 2536
2020-07-15 20:29:49,190 [INFO] 2537
2020-07-15 20:29:49,223 [INFO] 2538
2020-07-15 20:29:49,252 [INFO] 2539
2020-07-15 20:29:49,276 [INFO] 2540
2020-07-15 20:29:49,299 [INFO] 2542
2020-07-15 20:29:49,326 [INFO] 2543
2020-07-15 20:29:49,378 [INFO] 2545
2020-07-15 20:29:49,461 [INFO] 2546
2020-07-15 20:29:49,489 [INFO] 2547
2020-07-15 20:29:49,516 [INFO] 2548
2020-07-15 20:29:49,541 [INFO] 2596
2020-07-15 20:29:49,563 [INFO] 2597
2020-07-15 20:29:49,586 [INFO] 2601
2020-07-15 20:29:49,627 [INFO] 2603
2020-07-15 20:29:49,662 [INFO] 2605
2020-07-15 20:29:49,694 [INF

2020-07-15 20:29:56,816 [INFO] 3217
2020-07-15 20:29:56,840 [INFO] 3218
2020-07-15 20:29:56,863 [INFO] 3219
2020-07-15 20:29:56,874 [INFO] 3221
2020-07-15 20:29:56,898 [INFO] 3224
2020-07-15 20:29:56,922 [INFO] 3226
2020-07-15 20:29:56,946 [INFO] 3227
2020-07-15 20:29:56,970 [INFO] 3228
2020-07-15 20:29:57,005 [INFO] 3229
2020-07-15 20:29:57,053 [INFO] 3230
2020-07-15 20:29:57,104 [INFO] 3231
2020-07-15 20:29:57,149 [INFO] 3232
2020-07-15 20:29:57,174 [INFO] 3234
2020-07-15 20:29:57,197 [INFO] 3236
2020-07-15 20:29:57,220 [INFO] 3252
2020-07-15 20:29:57,246 [INFO] 3257
2020-07-15 20:29:57,270 [INFO] 3259
2020-07-15 20:29:57,312 [INFO] 3260
2020-07-15 20:29:57,335 [INFO] 3264
2020-07-15 20:29:57,359 [INFO] 3265
2020-07-15 20:29:57,386 [INFO] 3266
2020-07-15 20:29:57,414 [INFO] 3268
2020-07-15 20:29:57,444 [INFO] 3272
2020-07-15 20:29:57,472 [INFO] 3276
2020-07-15 20:29:57,499 [INFO] 3284
2020-07-15 20:29:57,527 [INFO] 3285
2020-07-15 20:29:57,555 [INFO] 3287
2020-07-15 20:29:57,583 [INF

2020-07-15 20:30:03,526 [INFO] 4137
2020-07-15 20:30:03,557 [INFO] 4138
2020-07-15 20:30:03,586 [INFO] 4139
2020-07-15 20:30:03,613 [INFO] 4141
2020-07-15 20:30:03,639 [INFO] 4142
2020-07-15 20:30:03,667 [INFO] 4144
2020-07-15 20:30:03,695 [INFO] 4147
2020-07-15 20:30:03,721 [INFO] 4148
2020-07-15 20:30:03,748 [INFO] 4152
2020-07-15 20:30:03,776 [INFO] 4153
2020-07-15 20:30:03,804 [INFO] 4154
2020-07-15 20:30:03,830 [INFO] 4155
2020-07-15 20:30:03,858 [INFO] 4157
2020-07-15 20:30:03,887 [INFO] 4160
2020-07-15 20:30:03,923 [INFO] 4161
2020-07-15 20:30:03,950 [INFO] 4162
2020-07-15 20:30:03,975 [INFO] 4163
2020-07-15 20:30:04,002 [INFO] 4164
2020-07-15 20:30:04,031 [INFO] 4167
2020-07-15 20:30:04,058 [INFO] 4168
2020-07-15 20:30:04,085 [INFO] 4171
2020-07-15 20:30:04,108 [INFO] 4173
2020-07-15 20:30:04,131 [INFO] 4174
2020-07-15 20:30:04,153 [INFO] 4175
2020-07-15 20:30:04,176 [INFO] 4180
2020-07-15 20:30:04,187 [INFO] 4183
2020-07-15 20:30:04,209 [INFO] 4188
2020-07-15 20:30:04,232 [INF

2020-07-15 20:30:09,949 [INFO] 5299
2020-07-15 20:30:09,976 [INFO] 5301
2020-07-15 20:30:10,000 [INFO] 5302
2020-07-15 20:30:10,025 [INFO] 5304
2020-07-15 20:30:10,048 [INFO] 5305
2020-07-15 20:30:10,072 [INFO] 5306
2020-07-15 20:30:10,096 [INFO] 5309
2020-07-15 20:30:10,128 [INFO] 5310
2020-07-15 20:30:10,157 [INFO] 5312
2020-07-15 20:30:10,191 [INFO] 5314
2020-07-15 20:30:10,225 [INFO] 5315
2020-07-15 20:30:10,259 [INFO] 5317
2020-07-15 20:30:10,273 [INFO] 5321
2020-07-15 20:30:10,305 [INFO] 5324
2020-07-15 20:30:10,336 [INFO] 5328
2020-07-15 20:30:10,362 [INFO] 5340
2020-07-15 20:30:10,395 [INFO] 5344
2020-07-15 20:30:10,417 [INFO] 5345
2020-07-15 20:30:10,440 [INFO] 5347
2020-07-15 20:30:10,461 [INFO] 5348
2020-07-15 20:30:10,485 [INFO] 5349
2020-07-15 20:30:10,507 [INFO] 5351
2020-07-15 20:30:10,528 [INFO] 5353
2020-07-15 20:30:10,551 [INFO] 5355
2020-07-15 20:30:10,573 [INFO] 5356
2020-07-15 20:30:10,595 [INFO] 5364
2020-07-15 20:30:10,617 [INFO] 5371
2020-07-15 20:30:10,638 [INF

2020-07-15 20:30:15,858 [INFO] 6233
2020-07-15 20:30:15,886 [INFO] 6234
2020-07-15 20:30:15,919 [INFO] 6235
2020-07-15 20:30:15,942 [INFO] 6236
2020-07-15 20:30:15,964 [INFO] 6237
2020-07-15 20:30:15,988 [INFO] 6238
2020-07-15 20:30:16,010 [INFO] 6239
2020-07-15 20:30:16,035 [INFO] 6240
2020-07-15 20:30:16,075 [INFO] 6241
2020-07-15 20:30:16,125 [INFO] 6242
2020-07-15 20:30:16,165 [INFO] 6243
2020-07-15 20:30:16,192 [INFO] 6244
2020-07-15 20:30:16,220 [INFO] 6245
2020-07-15 20:30:16,246 [INFO] 6246
2020-07-15 20:30:16,270 [INFO] 6247
2020-07-15 20:30:16,309 [INFO] 6248
2020-07-15 20:30:16,340 [INFO] 6251
2020-07-15 20:30:16,375 [INFO] 6257
2020-07-15 20:30:16,419 [INFO] 6259
2020-07-15 20:30:16,453 [INFO] 6261
2020-07-15 20:30:16,487 [INFO] 6263
2020-07-15 20:30:16,520 [INFO] 6264
2020-07-15 20:30:16,553 [INFO] 6265
2020-07-15 20:30:16,586 [INFO] 6266
2020-07-15 20:30:16,644 [INFO] 6269
2020-07-15 20:30:16,675 [INFO] 6270
2020-07-15 20:30:16,705 [INFO] 6271
2020-07-15 20:30:16,733 [INF

2020-07-15 20:30:21,986 [INFO] 8110
2020-07-15 20:30:22,012 [INFO] 8111
2020-07-15 20:30:22,035 [INFO] 8112
2020-07-15 20:30:22,057 [INFO] 8114
2020-07-15 20:30:22,079 [INFO] 8121
2020-07-15 20:30:22,100 [INFO] 8131
2020-07-15 20:30:22,122 [INFO] 8147
2020-07-15 20:30:22,148 [INFO] 8150
2020-07-15 20:30:22,174 [INFO] 8155
2020-07-15 20:30:22,196 [INFO] 8163
2020-07-15 20:30:22,220 [INFO] 8171
2020-07-15 20:30:22,249 [INFO] 8176
2020-07-15 20:30:22,275 [INFO] 8182
2020-07-15 20:30:22,298 [INFO] 8183
2020-07-15 20:30:22,319 [INFO] 8201
2020-07-15 20:30:22,341 [INFO] 8210
2020-07-15 20:30:22,365 [INFO] 8213
2020-07-15 20:30:22,387 [INFO] 8215
2020-07-15 20:30:22,409 [INFO] 8222
2020-07-15 20:30:22,431 [INFO] 8234
2020-07-15 20:30:22,453 [INFO] 8240
2020-07-15 20:30:22,474 [INFO] 8249
2020-07-15 20:30:22,496 [INFO] 8255
2020-07-15 20:30:22,518 [INFO] 8261
2020-07-15 20:30:22,539 [INFO] 8271
2020-07-15 20:30:22,561 [INFO] 8277
2020-07-15 20:30:22,585 [INFO] 8279
2020-07-15 20:30:22,606 [INF

In [20]:
pd.set_option('display.float_format', lambda x: '%.1f' % x)
alert_df[alert_df['日期']=='2020-07-15'].sort_values(['日期', '成交金額']).to_csv('20200715.csv', index=False)